<center><img src="car.jpg" width=500></center>


Insurance companies invest a lot of time and money into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries insurance it is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

(`Source: https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf`) 

Knowing all of this, On the Road car insurance have requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to identify the single feature that results in the best performing model, as measured by accuracy, so they can start with a simple model in production.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table detailing the column names and descriptions below.



## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [25]:
# Import required modules
import pandas as pd
import numpy as np
from statsmodels.formula.api import logit

# Start coding!
cars_insurance = pd.read_csv('car_insurance.csv')
# filling missing values
cars_insurance['credit_score'] = cars_insurance['credit_score'].fillna(cars_insurance['credit_score'].mean())
cars_insurance['annual_mileage'] = cars_insurance['annual_mileage'].fillna(cars_insurance['annual_mileage'].mean())
# Creating a list for the models
models = []
features = cars_insurance.columns.drop(['outcome', 'id'])

accuracies = []
conf_matrices = []
tn_list = []
tp_list = []
fn_list = []
fp_list = []

for col in features:
    # Check if the column is object (categorical), then convert to 'category'
    if cars_insurance[col].dtype == 'object':
        cars_insurance[col] = cars_insurance[col].astype('category')
    # Use Q(col) only if col has special characters, otherwise use col directly
    # Here, we use Q(col) for all to be safe
    model = logit(f'outcome ~ Q("{col}")', data=cars_insurance).fit(disp=0)
    models.append(model)
    
    # Confusion matrix
    conf_matrix = model.pred_table()
    conf_matrices.append(conf_matrix)
    tn = conf_matrix[0, 0]
    fp = conf_matrix[0, 1]
    fn = conf_matrix[1, 0]
    tp = conf_matrix[1, 1]
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    # Calculate accuracy from confusion matrix
    accuracy = (tn + tp) / conf_matrix.sum()
    accuracies.append(accuracy)

# Find the index of the best accuracy
best_accuracy_idx = np.argmax(accuracies)
best_feature = features[best_accuracy_idx]
best_accuracy_value = accuracies[best_accuracy_idx]
# Create a DataFrame with the best feature and its accuracy
best_feature_df = pd.DataFrame({'best_feature': [best_feature], 'best_accuracy': [best_accuracy_value]})

best_feature_df

,best_feature,best_accuracy
0,driving_experience,0.7771
